**Sustainable Software Development, block course, March 2021**  
*Scientific Software Center, Institute for Scientific Computing, Dr. Inga Ulusoy*

# Analysis of the data

Imagine you perform a "measurement" of some type and obtain "scientific data". You know what your data represents, but you have only a vague idea how different features in the data are connected, and what information you can extract from the data.

You would start first with going through the data, making sure your data set is complete and that the result is reasonable. Imagine this already happened.

In the next step, you would inspect your data more closely and try to identify structures. That is the step that we are focusing on in this unit.

In the `data` folder, you will find several data files (`*.t` and `*.dat`). These are data files generated through some "new approach" that hasn't been used in your lab before. No previous analysis software exists, and you are going to establish a protocol for this "new approach" and "publish your results".

The data can be grouped into two categories: 
1. data to be analyzed using statistical methods;
2. data to be analyzed using numerical methods.

In your hypothetical lab, you are an "expert" in one particular "method", and your co-worker is an "expert" in the other. Combined these two methods will lead to much more impactful results than if only one of you analyzed the data. Now, the task in this course is to be solved collaboratively with your team member working on one of the analysis approaches, and you working on the other. You will both implement functionality into the same piece of "software", but do so collaboratively through git.

As you do not know yet which analysis is most meaningful for your data, and how to implement it, you will start with a jupyter notebook. You and your team member will work on the same notebook that will be part of a github repository for your project. This is the task for today. Discuss with your team members who will work on the statistical and who on the numerical analysis.

## Step 1

Generate a github repository with the relevant files.

## Step 2

Clone the repository to your local machine.

## Step 3

Start working on task 1 for your analysis approach. 

## Step 4

Create your own branch of the repository and commit your changes to your branch; push to the remote repository.

## Step 5

Open a `pull request` so your team member can review your implementation. Likewise, your team member will ask you to review theirs.

## Step 6

Merge the changes in your branch into `main`. Resolve conflicts.

## Step 7

Repeat working on task; committing and pushing to your previously generated branch or a new branch; open a pull request; merge with main; until you have finished all the tasks in your analysis approach. Delete obsolete branches.

# Start of the analysis notebook

**Author : Team 09: Krishna, Robin and Sudip**  
*Date : 11.03.2021*  
*Affiliation : Heidelberg University*  

Place the required modules in the top, followed by required constants and global functions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
from os import path

In [ ]:
# constants and global functions
threshv = 1.0e-5
filenames = 'efield.t', 'expec.t', 'npop.t', 'nstate_i.t', 'table.dat'
filedir = '../data/'
plotdir = '../plots/'

In [ ]:
# reading of the data files

#for pandas dataframe
def read_df(filedir, filename):
    name = '{}{}'.format(filedir, filename)
    print('Reading from file {} - pandas'.format(name))
    data = pd.read_csv(name, r'\s+')
    return data

#for numpy array
def read_np(filedir, filename):
    name = '{}{}'.format(filedir, filename)
    print('Reading from file {} - numpy'.format(name))
    data = np.loadtxt(name, skiprows=1)
    data = data.T
    return data

#read every file as pandas DataFrame
#files_pd = [read_df(filedir, fn) for fn in filenames]
#read every file as numpy array
#files_np = [read_np(filedir, fn) for fn in filenames]

In [ ]:
#a function to save a seaborn plot
def save_plot(figure, plotdir, filename):
    fig = figure.get_figure()
    print('Saving figure at {}{}'.format(plotdir,filename))
    if not path.exists(plotdir):
        os.mkdir(plotdir)
    fig.savefig('{}{}'.format(plotdir,filename))
    return


# Statistical analysis

Find correlations in the data sets. Analyse the data statistically and plot your results.  

Here we would want to do everything with pandas and leave the data in a dataframe. The files that are relevant to you are `expect.t`, `npop.t` and `table.dat`.

### Task 1: Read in expec.t and plot relevant data

In [ ]:
# read and plot expec.t
expect = read_df(filedir, filenames[1])
expect.head()



In [ ]:
sn.lineplot(data = expect, x = "time", y = "<z>")

In [ ]:
sn.lineplot(data = expect, x = "time", y = "<H>")

In [ ]:
sn.lineplot(data = expect, x = "<H>", y = "<z>")

We can discard the entries norm, \<x>, and \<y> as these are mostly constant.

In [ ]:
# eliminate columns based on the variance: if the variance of the values
# in a column is below a given threshold, that column is discarded
def remove_vars_with_low_variance(df, threshv):
    variables_below_var_thresh = df.var()[df.var() < threshv].index.values
    return df.drop(variables_below_var_thresh, axis = 1)

expect_reduced = remove_vars_with_low_variance(expect, threshv)
expect_reduced

### Task 2: Create plots of the relevant data and save as .pdf.

In [ ]:
# create plots
z_time_plot = sn.lineplot(data = expect_reduced, x = "time", y = "<z>")
save_plot(z_time_plot, plotdir, "expect_z_over_time.pdf")



In [ ]:
H_time_plot = sn.lineplot(data = expect_reduced, x = "time", y = "<H>")
save_plot(z_time_plot, plotdir, "expect_H_over_time.pdf")

### Task 3: Read in file `npop.t` and analyze correlations in the data

In [ ]:
 # read in npop.t
npopt = read_df(filedir, filenames[2])
npopt.head()

In [ ]:
# discard all columns with variance below a set threshold - we can consider them as constant
npopt_reduced = remove_vars_with_low_variance(npopt, threshv)

In [ ]:
#transform from wide to long format
npopt_reduced_long = pd.melt(npopt_reduced, id_vars="time")
npopt_reduced_long.head()

Plot the remaining columns. Seaborn prefers "long format" (one column for all measurement values, one column to indicate the type) as input, whereas the cvs is in "wide format" (one column per measurement type).

In [ ]:
# plot with seaborn from long format
pl = sn.lineplot(data = npopt_reduced_long, x = "time", y = "value", hue = "variable")
save_plot(pl, plotdir, "npopt_time_course.pdf")

## Quantify the pairwise correlation in the data

- negative correlation: y values decrease for increasing x - large values of one feature correspond to small values of the other feature
- weak or no correlation: no trend observable, association between two features is hardly observable
- positive correlation: y values increase for decreasing x - small values of one feature correspond to small values of the other feature

Remember that correlation does not indicate causation - the reason that two features are associated can lie in their dependence on same factors.

Correlate the value pairs using Pearson's $r$. Pearson's $r$ is a measure of the linear relationship between features:

$r = \frac{\sum_i(x_i − \bar{x})(y_i − \bar{y})}{\sqrt{\sum_i(x_i − \bar{x})^2 \sum_i(y_i − \bar{y})^2}}$

Here, $\bar{x}$ and $\bar{y}$ indicate mean values. $i$ runs over the whole data set. For a positive correlation, $r$ is positive, and negative for a negative correlation, with minimum and maximum values of -1 and 1, indicating a perfectly linear relationship. Weakly or not correlated features are characterized by $r$-values close to 0.

Other measures of correlation that can be used are Spearman's rank (value pairs follow monotonic function) or Kendall's $\tau$ (measures ordinal association), but they do not apply here. You can also define measures yourself.

In [ ]:
# print the correlation matrix
npopt_wo_time = npopt_reduced.drop(["time"], axis = 1)
cm = npopt_wo_time.corr(method = "pearson")
sn.heatmap(cm)

The diagonal values tell us that each value is perfectly correlated with itself. We are not interested in the diagonal values and also not in the correlation with time. We also need to get rid of redundant entries. Finally, we need to find the value pairs that exhibit the highest linear correlation. We still want to know if it is positive or negative correlation, so we cannot get rid of the sign.

In [ ]:
# get rid of lower triangular and diagonal entries of the correlation matrix
def upper_triangle(mat):
    sh = mat.shape
    for i in range(sh[0]):
        for j in range(sh[1]):
            if i >= j:
                mat.iloc[i,j] = np.nan
    return mat
cm = upper_triangle(cm)

hm = sn.heatmap(cm)
save_plot(hm, plotdir, "correlation_heatmap.pdf")

In [ ]:
# sort the remaing values according to their absolute value, but keep the sign
def sort_corrs_by_abs_val(mat):
    #helper column
    mat["sample"] = mat.index
    #flatten df
    mat_flat = mat.melt(id_vars = "sample")
    #sort out the unneccessary (which are nan) 
    idx_to_drop = mat_flat.index[np.isnan(mat_flat.value)].values
    mat_flat = mat_flat.drop(idx_to_drop,axis = 0)
    #helper column for absolute values
    mat_flat["abs_val"] = abs(mat_flat.value)
    mat_flat = mat_flat.sort_values(by="abs_val", ascending=False)
    return mat_flat

sorted_corrs = sort_corrs_by_abs_val(cm)

#put descriptive names
sorted_corrs = sorted_corrs.rename(columns = {"sample":"sample1","variable":"sample2", "value":"corr_coeff", "abs_val":"abs_corr_coeff"})
sorted_corrs.head()

Note that the entries in the left column are not repeated if they do not change from the row above (so the fourth feature pair is MO3 and MO6).

### Task 4: Print the resulting data to a file

In [ ]:
# write to file
def save_df(df, filedir, filename):
    print('Saving data at {}{}'.format(filedir,filename))
    df.to_csv('{}{}'.format(filedir, filename), index=False)
    return

filename = "npopt_correlations_sorted.csv"
save_df(sorted_corrs, filedir, filename)

### Task 5: Calculate the Euclidean distance (L2 norm) for the vectors in `table.dat`


The Euclidean distance measures the distance between to objects that are not points:

$d(p,q) = \sqrt{\left(p-q\right)^2}$

In this case, consider each of the columns in table.dat as a vector in Euclidean space, where column $r(x)$ and column $v(x)$ denote a pair of vectors that should be compared, as well as $r(y)$ and $v(y)$, and r(z) and v(z).

(Background: These are dipole moment components in different gauges, the length and velocity gauge.)

In [ ]:
# read in table.dat - I suggest reading it as a numpy array
table = read_np(filedir, filenames[4])

# replace the NaNs by zero
table[np.isnan(table)] = 0





Now calculate how different the vectors in column 2 are from column 3, column 4 from column 5, and column 6 from column 7.

In [ ]:
# calculate the Euclidean distance
def euclid_distance_between_rows(array,index1,index2):
    v1 = array[index1,:]
    v2 = array[index2,:]
    return np.linalg.norm(v1 - v2)
    

rows1 =[2,4,6]
rows2 = [3,5,7]
distances = np.zeros_like(rows1)

for i, i1,i2 in zip(range(len(rows1)),rows1, rows2):
    distances[i] = euclid_distance_between_rows(table, i1,i2)

In [ ]:
# plot the result and save to a .pdf
pl = sn.barplot(x = ["x-direction", "y-direction", "z-direction"], y = distances)
pl.set_title("Euclidian Distance in all spatial dimensions")
pl.set_ylabel("D")

filename = "eucl_distances_in_3_dims.pdf"
save_plot(pl,plotdir, filename)

In [ ]:
# print the result to a file
def save_np_to_file(nparray, filedir, filename):
    name = "{}{}".format(filedir, filename)
    print("Saving data to {}".format(name))
    np.savetxt(fname = name, X = nparray)
    return
filename = "eucl_distances_in_3_dims.out"
save_np_to_file(distances, filedir, filename)

# Numerical analysis

Analyze the data using autocorrelation functions and discrete Fourier transforms. Plot your results.

### Task 1: Read in `efield.t` and Fourier-transform relevant columns

In [ ]:
# read and plot efield.t
efield = read_df(filedir, filenames[0])
sn.pairplot(efield)

Here we are interested in column 2 since the others are constant.

In [ ]:
# discard the columns with variance below threshold - these are considered constant
efield_reduced = remove_vars_with_low_variance(efield, threshv)
efield_reduced.head()

In [ ]:
# discrete Fourier transform of the remaining column: You only need the real frequencies
def dft_real_df(data):
    data_s = np.fft.rfft(data)
    data_w = np.fft.rfftfreq(len(data.index))
    return data_s, data_w
efield_s, freq = dft_real_df(efield_reduced["y"])


### Task 2: Generate a plot of your results to be saved as pdf.

In [ ]:
# plot your results
pl = sn.lineplot(x = freq, y = abs(efield_s)**2)
pl.set_title("FT of efield in y")
save_plot(pl, plotdir, "FT_efield_y.pdf")

### Task 3: Calculate the autocorrelation function from nstate_i.t
The autocorrelation function measures how correlated subsequent vectors are with an initial vector; ie. 

$\Psi_{corr} = \langle \Psi(t=0) | \Psi(t) \rangle = \int_0^{tfin} \Psi(0)^* \Psi(t) dt$

Since we are in a numerical representation, the integral can be replaced with a sum; and the given vectors are already normalized.

In [ ]:
# read in as numpy array
nstate = read_np(filedir, filenames[3])

In [ ]:
# store the time column (column 0) in a vector and drop from array
time = nstate[:,0]
nstate = np.delete(nstate, 0, axis = 0)
nstate.shape

In [ ]:
# correct the data representation: this is in fact a complex matrix
# the real part of each matrix column is contained in numpy array column 0, 2, 4, 6, ...
# the imaginary part of each matrix column is contained in numpy array column 1, 3, 5, 7, ...
# convert the array that was read as dtype=float into a dtype=complex array

def correct_matrix(mat):
    r_parts = mat[0::2]
    i_parts = mat[1::2]
    return r_parts+ 1j* i_parts
    
corr_mat = correct_matrix(nstate)

In [ ]:
# for the autocorrelation function, we want the overlap between the first vector at time 0 and all 
# subsequent vectors at later times - the sum of the product of initial and subsequent vectors for each time step

def calc_auto(wavef):
    aucofu = np.zeros(len(wavef[0]),dtype = complex)
    for i in range(0,len(wavef[0])):
        aucofu[i] = np.sum(wavef[:,0]*np.conjugate(wavef[:,i]))
    return aucofu

aucofu = calc_auto(corr_mat)
plt.plot(abs(aucofu**2))
plt.show()

### Task 4: Generate a plot of your results to be saved as pdf.

In [ ]:
# plot the autocorrelation function - real, imaginary and absolute part
def plot_aucofu(aucofu):
    r = np.real(aucofu)
    i = np.imag(aucofu)
    a = abs(aucofu**2)
    df = pd.DataFrame({"real":r, "imag":i, "absolute":a})
    df["time"] = df.index
    df = df.melt(id_vars= "time")
    pl = sn.lineplot(data = df, x = "time", y = "value", hue = "variable")
    pl.set_ylabel("Autocorrelation")
    return pl

pl = plot_aucofu(aucofu)
name = "autocorrelation_function_nstate.pdf"
save_plot(pl, plotdir, name)

### Task 5: Discrete Fourier transform of the autocorrelation function

In [ ]:
# discrete Fourier-transform the autocorrelation function - now we need all frequency components, 
# also the negative ones
def dft_np(array):
    data_s = np.fft.fft(array)
    freqs = np.fft.fftfreq(array.shape[0])
    return data_s, freqs
aucofu_ft, freq = dft_np(aucofu)

### Task 6: Generate a plot of your results to be saved as pdf.

In [ ]:
# plot the power spectrum (abs**2)
pl = sn.lineplot(x = freq, y = abs(aucofu_ft)**2)
pl.set_ylim(-0.1,15)
pl.set_xlabel("Frequency")
pl.set_ylabel("Energy")
name = "power_spectrum_of_aucofu.pdf"
save_plot(pl, plotdir, name)